In [67]:
import pandas as pd
from gensim import models, corpora
from gensim.parsing.preprocessing import preprocess_string, strip_tags, strip_punctuation
from gensim.parsing.preprocessing import remove_stopwords, stem_text, strip_non_alphanum, strip_multiple_whitespaces
from gensim.parsing.preprocessing import strip_short, strip_numeric
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import nltk
import pandas as pd
import numpy as np
import tensorflow as tf
import parmap
import swifter
from tqdm import tqdm

In [68]:
topics_index_to_name_map = {
    0: 'Agriculture, animals, food and rural affairs',
    1: 'Asylum, immigration and nationality',
    2: 'Business, industry and consumers',
    3: 'Communities and families',
    4: 'Crime, civil law, justice and rights',
    5: 'Culture, media and sport',
    6: 'Defence',
    7: 'Economy and finance',
    8: 'Education',
    9: 'Employment and training',
    10: 'Energy and environment',
    11: 'European Union',
    12: 'Health services and medicine',
    13: 'Housing and planning',
    14: 'International affairs',
    15: 'Parliament, government and politics',
    16: 'Science and technology',
    17: 'Social security and pensions',
    18: 'Social services',
    19: 'Transport',
    20: 'Others'
}
topics_name_to_index_map = {y:x for x,y in topics_index_to_name_map.items()}

def strip_short2(text):
    return strip_short(text, minsize=4)


def preprocess_text(text):
    FILTERS = [lambda x: x.lower(), strip_multiple_whitespaces, strip_tags, strip_punctuation,
                   strip_non_alphanum, strip_numeric, strip_short2]
    return preprocess_string(text, FILTERS)

def preprocess(topic):
    ret = []
    topic = topic.strip()
    
    if '|' in topic:
        topics = topic.split('|')
        t = topics[0]
        t = t.strip()
        return topics_name_to_index_map[t]
        
    return topics_name_to_index_map[topic]

In [110]:
def one_hot(x):
    x_ = [0] * len(x)
    x_[np.argmax(x)] = 1
    
    return x_

def predict(X):
    preprocessed_X = preprocess_text(X)
    vec_X = doc2vec_model.infer_vector(preprocessed_X)
    vec_X = vec_X.reshape(1, vec_X.shape[0])
    pred = m0.predict(vec_X)[0]
    g = enc0.inverse_transform([one_hot(pred)])[0]
    
    if g == 1:
        pred = m1.predict(vec_X)[0]
        pred = enc1.inverse_transform([one_hot(pred)])[0][0]
        if pred == 4:
            pred = m4.predict(vec_X)[0]
            pred = enc4.inverse_transform([one_hot(pred)])[0][0]
            return topics_index_to_name_map[pred]
        elif pred == 5:
            pred = m5.predict(vec_X)[0]
            pred = enc5.inverse_transform([one_hot(pred)])[0][0]
            return topics_index_to_name_map[pred]
        elif pred == 6:
            pred = m6.predict(vec_X)[0]
            pred = enc6.inverse_transform([one_hot(pred)])[0][0]
            return topics_index_to_name_map[pred]
        elif pred == 7:
            pred = m7.predict(vec_X)[0]
            pred = enc7.inverse_transform([one_hot(pred)])[0][0]
            return topics_index_to_name_map[pred]
    elif g == 2:
        pred = m2.predict(vec_X)[0]
        pred = enc2.inverse_transform([one_hot(pred)])[0][0]
        return topics_index_to_name_map[pred]
    elif g == 3:
        pred = m3.predict(vec_X)[0]
        pred = enc3.inverse_transform([one_hot(pred)])[0][0]
        return topics_index_to_name_map[pred]
    
def build_network(enc):
    model = tf.keras.models.Sequential([
        tf.keras.Input(shape=(100)),
        tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l=0.1)),
        tf.keras.layers.Dense(len(enc.categories_[0]), activation='softmax')
    ])
    loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=False, label_smoothing=0, name='categorical_crossentropy')

    model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=loss_fn,
              metrics=['accuracy'])
    
    return model

In [81]:
doc2vec_model = Doc2Vec.load('models/doc2vec/doc2vec_2010_2016')

enc0 = load('./models/doc2vec/encoder_{}_{}_{}.joblib'.format('0', 2010, 2016))
enc1 = load('./models/doc2vec/encoder_{}_{}_{}.joblib'.format('1', 2010, 2016))
enc2 = load('./models/doc2vec/encoder_{}_{}_{}.joblib'.format('2', 2010, 2016))
enc3 = load('./models/doc2vec/encoder_{}_{}_{}.joblib'.format('3', 2010, 2016))
enc4 = load('./models/doc2vec/encoder_{}_{}_{}.joblib'.format('4', 2010, 2016))
enc5 = load('./models/doc2vec/encoder_{}_{}_{}.joblib'.format('5', 2010, 2016))
enc6 = load('./models/doc2vec/encoder_{}_{}_{}.joblib'.format('6', 2010, 2016))
enc7 = load('./models/doc2vec/encoder_{}_{}_{}.joblib'.format('7', 2010, 2016))

m0 = build_network(enc0)
m0.load_weights('./models/doc2vec/classifier_{}_2010_2016'.format('0'))

m1 = build_network(enc1)
m1.load_weights('./models/doc2vec/classifier_{}_2010_2016'.format('1'))

m2 = build_network(enc_g2)
m2.load_weights('./models/doc2vec/classifier_{}_2010_2016'.format('2'))

m3 = build_network(enc_g3)
m3.load_weights('./models/doc2vec/classifier_{}_2010_2016'.format('3'))

m4 = build_network(enc_g4)
m4.load_weights('./models/doc2vec/classifier_{}_2010_2016'.format('4'))

m5 = build_network(enc_g5)
m5.load_weights('./models/doc2vec/classifier_{}_2010_2016'.format('5'))

m6 = build_network(enc_g6)
m6.load_weights('./models/doc2vec/classifier_{}_2010_2016'.format('6'))

m7 = build_network(enc_g7)
m7.load_weights('./models/doc2vec/classifier_{}_2010_2016'.format('7'))

In [75]:
df = pd.read_csv('./data/2017_debate.csv')
df = df.drop(df[df.topic == 'admin'].index)
df = df.drop(df[df.transcript.str.split().map(len) < 10].index).reset_index()
df['topic'] = df.apply(lambda row: preprocess(row['topic']), axis=1)

In [76]:
df

,index,topic,transcript
0,0,20,"In the previous Parliament, we took action to ..."
1,1,19,"I am grateful to you, Madam Deputy Speaker, an..."
2,2,10,(Urgent Question): To ask the Secretary of Sta...
3,3,20,"As I am often moved to observe, and I observe ..."
4,4,6,"I beg to move, That the Bill be now read a Sec..."
...,...,...,...
1739,1777,9,3. What assessment his Department has made of ...
1740,1778,9,7. What recent assessment he has made of trend...
1741,1779,9,3. What progress has been made on implementing...
1742,1780,9,"I beg to move, That leave be given to bring i..."


In [112]:
X = df['transcript'].values
Y = df['topic'].values
i = 113
print('predicted ', predict(X[i]))
print('true: ', topics_index_to_name_map[Y[i]])

predicted  Crime, civil law, justice and rights
true:  Crime, civil law, justice and rights


In [114]:
predictions = []

for x in tqdm(X):
    predictions.append(predict(x))

df['hierarichy_prediction'] = predictions

100%|██████████| 1744/1744 [09:00<00:00,  3.23it/s]


In [115]:
df

,index,topic,transcript,hierarichy_prediction
0,0,20,"In the previous Parliament, we took action to ...",Health services and medicine
1,1,19,"I am grateful to you, Madam Deputy Speaker, an...",Transport
2,2,10,(Urgent Question): To ask the Secretary of Sta...,Energy and environment
3,3,20,"As I am often moved to observe, and I observe ...",Others
4,4,6,"I beg to move, That the Bill be now read a Sec...",Defence
...,...,...,...,...
1739,1777,9,3. What assessment his Department has made of ...,Education
1740,1778,9,7. What recent assessment he has made of trend...,International affairs
1741,1779,9,3. What progress has been made on implementing...,Employment and training
1742,1780,9,"I beg to move, That leave be given to bring i...",Others


In [116]:
df['topic'] = df['topic'].apply(lambda x: topics_index_to_name_map[x])
df

,index,topic,transcript,hierarichy_prediction
0,0,Others,"In the previous Parliament, we took action to ...",Health services and medicine
1,1,Transport,"I am grateful to you, Madam Deputy Speaker, an...",Transport
2,2,Energy and environment,(Urgent Question): To ask the Secretary of Sta...,Energy and environment
3,3,Others,"As I am often moved to observe, and I observe ...",Others
4,4,Defence,"I beg to move, That the Bill be now read a Sec...",Defence
...,...,...,...,...
1739,1777,Employment and training,3. What assessment his Department has made of ...,Education
1740,1778,Employment and training,7. What recent assessment he has made of trend...,International affairs
1741,1779,Employment and training,3. What progress has been made on implementing...,Employment and training
1742,1780,Employment and training,"I beg to move, That leave be given to bring i...",Others


In [130]:
df.to_csv('debate_predictions_2017.csv')

In [129]:
df.iloc[1740]['transcript']

'7. What recent assessment he has made of trends in youth unemployment.  [902984]  Unemployment among 16 to 24-year-olds is 523,000—down 60,000 on the year and down 416,000 since 2010.  I welcome that news from the Minister. I am a strong campaigner for apprenticeships, including in my constituency, where we have just 70 young unemployed people. Does the Minister agree that making apprenticeships far more available helps young people into jobs, not only in Wealden but throughout the country?  My hon. Friend has indeed been a great campaigner and a great champion for apprenticeships. Apprenticeships—including the 620 starts in Wealden in 2016-17—are one of the key policies that have contributed to our successful labour market, in which employment now stands at 75%.  What is the trend in unemployment for young people with learning difficulties?  We absolutely accept that of course young people with learning difficulties need additional assistance and additional understanding of condition

In [118]:
counts = df['topic'].value_counts()
counts

Crime, civil law, justice and rights            227
Parliament, government and politics             217
European Union                                  134
International affairs                           129
Health services and medicine                    127
Communities and families                        116
Transport                                       107
Economy and finance                             104
Education                                        82
Employment and training                          78
Culture, media and sport                         77
Business, industry and consumers                 72
Others                                           71
Defence                                          70
Energy and environment                           45
Agriculture, animals, food and rural affairs     29
Asylum, immigration and nationality              16
Housing and planning                             15
Social security and pensions                     12
Science and 

In [125]:
topics = topics_index_to_name_map.values()
accurate_pred = {topic:0 for topic in topics}
total_true = 0

In [126]:
for index, row in df.iterrows():
    if row['topic'] == row['hierarichy_prediction']:
        accurate_pred[row['topic']] += 1
        total_true += 1

In [121]:
accurate_pred

{'Agriculture, animals, food and rural affairs': 20,
 'Asylum, immigration and nationality': 6,
 'Business, industry and consumers': 40,
 'Communities and families': 55,
 'Crime, civil law, justice and rights': 181,
 'Culture, media and sport': 48,
 'Defence': 55,
 'Economy and finance': 43,
 'Education': 70,
 'Employment and training': 50,
 'Energy and environment': 35,
 'European Union': 31,
 'Health services and medicine': 109,
 'Housing and planning': 5,
 'International affairs': 84,
 'Parliament, government and politics': 156,
 'Science and technology': 0,
 'Social security and pensions': 3,
 'Social services': 1,
 'Transport': 87,
 'Others': 6}

In [122]:
rows = []

for topic in topics:
    rows.append([topic, accurate_pred[topic]/ counts[topic]])
    
res = pd.DataFrame(rows, columns=['topic', 'acc'])

In [123]:
res

,topic,acc
0,"Agriculture, animals, food and rural affairs",0.689655
1,"Asylum, immigration and nationality",0.375000
2,"Business, industry and consumers",0.555556
3,Communities and families,0.474138
4,"Crime, civil law, justice and rights",0.797357
5,"Culture, media and sport",0.623377
6,Defence,0.785714
7,Economy and finance,0.413462
8,Education,0.853659
9,Employment and training,0.641026


In [131]:
res.to_csv('accuracy_2017.csv')

In [127]:
total_true/ len(df)

0.6221330275229358